In [1]:
# fopen = open('dumping-common-crawl.txt')
# results = [next(fopen) for _ in range(1000000)]
# results[:1000]

In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/common-crawl/mse-index.zip
# !unzip mse-index.zip

In [3]:
import pandas as pd

In [4]:
# df = pd.read_feather('data/cdx-00017.feather')

In [5]:
from glob import glob
files = glob('feather/*.feather')
len(files)

55982

In [6]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/common-crawl/feather.zip
# !unzip feather

In [7]:
import dask
import feather
import dask.dataframe as dd
import dask.bag as db
import itertools

dask.__version__

'2.25.0'

In [8]:
# from dask.distributed import Client

# client = Client(n_workers=16)
# client

# dfs = [dask.delayed(pd.read_feather)(f) for f in files]
# df = dd.from_delayed(dfs)

In [9]:
%%time

def get_tld(file):
    df = pd.read_feather(file)
    return df.groupby('tld')['sentence'].apply(list).reset_index()

bags = db.from_sequence(files)
mapped = bags.map(get_tld)
computed = mapped.compute(num_workers = 20)

CPU times: user 51.7 s, sys: 5.4 s, total: 57 s
Wall time: 1min 2s


In [13]:
computed[0]

,tld,sentence
0,com,"[nurfarahin za'ba, Tiada catatan., Laman utama..."
1,com.my,[Projek - Konsortium Perumahan Rakyat Terengga...
2,gov.my,[Perolehan Darah - Laman Web Rasmi Pusat Darah...
3,ml,"[Bapa rogol anak jepun seks lucah, seks video-..."


In [15]:
import itertools

def nested_list(rows):
    merged = list(itertools.chain(*rows))
    merged = list(filter(None, merged))
    merged = list(set(merged))
    return merged

concated = pd.concat(computed)
agg = concated.groupby('tld').agg({'sentence': nested_list})

In [27]:
for i in range(len(agg)):
    print(i, agg.index[i], agg['sentence'].iloc[i][:10], '\n')

0 ac.at ['español', 'Contact', 'An email will be sent to you to confirm the registration.', 'Create event', 'Apache/2.4.25 (Debian) Server at ftp.tu-graz.ac.at Port 80', 'English', '* First name', 'This is not a conference registration form but an Indico account creation.', 'After the submission of your personal data, an email will be sent to you.', 'You must enter a valid email address.'] 

1 ac.id ["Dosen Hasyim Asy'ari", 'saiful200488@gmail.com', 'perseorangan.', 'Pengambilan KRS', 'Kediaman Rasulullah', '• Institusi ekonomi.', "Al-Ma'idah", 'Begitu pula, beliau tidak lupa berterima kasih kepada orang yang menghidangkannya serta tidak mencela bila ada hal yang kurang berkenan.', '39 2016/2017 Genap TAFSIR 4 2 A Dirasat Islamiyah Dirasat Islamiyah', '38 2017/2018 Ganjil PEMIKIRAN POLITIK ISLAM KONTEMPORER 2 HTN Syariah dan Hukum Hukum Tata Negara (Siyasah)'] 

2 ac.in ['E-mail : vspvmadhavan@gmail.com', 'S COBBLESTONE ST,GILBERT', 'KUWAITI DANISH DIARY CO,', 'F B AREA,KARACHI 38', 'L

In [ ]:
archi, asia, at, audio, ax, band, bc.ca, bg, bid, biz, biz.id, blog, business, bz
ca, cafe, cat, cc, center, cf, ch, church, city, cl, click, cloud, cm, cn,
co, co.id, co.in, co.jp, co.kr, co.nz, com, com.cau, com.bn, com.br, com.cn,
com.co, com.cy, com.mx, com.my, com.pl, com.py, coop, cz, desa.id, digital,
direct, edu, edu.cn, edu.my, ee, es, eu, fi, fm, fr, fun, ga, global, go.id, go.jp,
gov, gov.bn, gov.mo, gov.my, gq, gr, graphics, gratis, help, hk, hu, im, international,
io, ir, is, it, jp, kg, la, lg.jp, li, link, live, lt, lv, ma, men, mil.id, mil.my,
